In [59]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

## Summary

#### Results
1. Best performed model is NN with roc_auc_score of 0.8965 on Chris's and my features.
2. NN model is saved as 'best_model2.h5'.
3. My hypothesis on synthetic drugs doesn't work. I can probably try PCA to improve the 65_brands features(?)

#### Next step
1. Try PCA on features_matthew_65brands and try to run again.
2. Further tune NN, RF.
3. Try Gradient Boosting.
4. Ensemble these models.

#### Data and models
1. Features: features_matthew_2, pc_diagnosis, features_matthew_65brands(useless)
2. Labels: response_variable_v2
3. Models: logistic regression, RF, Neural Network, KNN(useless)

#### Structure of code
1. Process input and output
2. Models

#### Issue to be discussed
1. In hold_out set, there are [16] ppl has no supply_day. How should we treet them?
2. In train data, we have 129 problematic ppl: 
3. 18 of them have no supply_day data when day >= 0
4. 34 of them have no supply_day data through out all period
5. 68 of them have no supply_day data when day > 0
6. 9 of them have no diagnosis data (but have supply_day data)

#### Others
1. Make sure make probability prediction when calculating roc_auc_score.
2. Ref: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html


## Process input and outpur

In [2]:
%%time
df = pd.read_csv('C:/Users/spong/Desktop/Humana/HMAHCC_COMP.csv')

rx_paid = df[df['event_descr']=='RX Claim - Paid']

rx_paid = rx_paid.drop(['event_attr2',
                        'event_attr7'],
                         axis = 1)

rx_paid.columns = ['id',
                   'event_descr',
                   'gpi_drug_class_description',
                   'rx_cost',
                   'net_paid_amount',
                   'brand_name',
                   'drug_group_description',
                   'generic_name',
                   'member_responsible_amount',
                   'gpi_drug_group8_id',
                   'Days',
                   'PAY_DAY_SUPPLY_CNT',
                   'PAYABLE_QTY',
                   'MME',
                   'DRUG_TYPE',
                   'Specialty',
                   'Specialty2',
                   'Specialty3']

opioid_col = ['id',
              'gpi_drug_class_description',
              'brand_name',
              'drug_group_description',
              'generic_name',
              'gpi_drug_group8_id',
              'PAY_DAY_SUPPLY_CNT',
              'PAYABLE_QTY',
              'MME',
              'DRUG_TYPE',
              'Specialty',
              'Specialty2',
              'Specialty3',
              'Days']

opioid = rx_paid[(rx_paid['PAY_DAY_SUPPLY_CNT'].notnull())&(rx_paid['Days']<=0)][opioid_col]

<string>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 13.3 s


## Failed attemp to fill missing values on supply_CNT and MME
Supply_CNT means opoid, but opioid doesn't mean it has supply_CNT data
Use generic_name

In [3]:
# opioid_all_time is data that generic_name with 
opioid_all_time = rx_paid[rx_paid['PAY_DAY_SUPPLY_CNT'].notnull()]['generic_name'].value_counts()
print('total numbers of opioid generic_name', len(opioid_all_time))

total numbers of opioid generic_name 213


In [4]:
mask = rx_paid['generic_name'].map(lambda x: x in opioid_all_time.index.values)

# true_opioid: rx_paid data filtered by generic_names is in opioid_all_time
true_opioid = rx_paid[mask]

In [ ]:
# true_opioid without Supply_CNT
true_opioid_Supply_CNT = rx_paid[mask][rx_paid[mask]['PAY_DAY_SUPPLY_CNT'].notnull()]
# true_opioid without Supply_CNT
true_opioid_no_Supply_CNT = rx_paid[mask][rx_paid[mask]['PAY_DAY_SUPPLY_CNT'].isnull()]

#### Missing values in PAY_DAY_SUPPLY_CNT
2.05% of opioid drug data don't have PAY_DAY_SUPPLY_CNT data

In [22]:
all_counts = true_opioid_Supply_CNT[true_opioid_Supply_CNT['Days']==0].shape[0]
missing_counts = true_opioid_no_Supply_CNT[true_opioid_no_Supply_CNT['Days']==0].shape[0]

missing_counts/all_counts

0.02052119576794717

In [20]:
tos_gn = true_opioid_Supply_CNT[true_opioid_Supply_CNT['Days']==0]['generic_name'].drop_duplicates()
tons_gn = true_opioid_no_Supply_CNT[true_opioid_no_Supply_CNT['Days']==0]['generic_name'].drop_duplicates()

[x for x in tos_gn.values if x in tons_gn.values][:5]

['TRAMADOL HCL TAB 50 MG',
 'OXYCODONE W/ ACETAMINOPHEN TAB 5-325 MG',
 'OXYCODONE HCL TAB 10 MG',
 'HYDROCODONE-ACETAMINOPHEN TAB 5-325 MG',
 'HYDROCODONE-ACETAMINOPHEN TAB 10-325 MG']

In [33]:
mask2 = (true_opioid['generic_name'] == 'HYDROCODONE-ACETAMINOPHEN TAB 10-325 MG')&(true_opioid['Days']<=0)
true_opioid[mask2]['PAY_DAY_SUPPLY_CNT'].mode()

0    30.0
dtype: float64

#### IDs with missing value in PAY_DAY_SUPPLY_CNT
85 ppl or 0.6% of ppl had opioid drugs of day<=0 but without PAY_DAY_SUPPLY_CNT data

In [15]:
tos_id = true_opioid_Supply_CNT[true_opioid_Supply_CNT['Days']<=0]['id'].drop_duplicates()
tons_id = true_opioid_no_Supply_CNT[true_opioid_no_Supply_CNT['Days']<=0]['id'].drop_duplicates()

len([x for x in tons_id.values if x not in tos_id.values])
# 85 people have opioid data but no supply_day data on day<=0

85

In [35]:
85/14000

0.006071428571428571

17 ppl have no opioid drugs data

In [66]:
df_id = df['id'].drop_duplicates() 
true_opioid_id = true_opioid[true_opioid['Days']<=0]['id'].drop_duplicates() 

nouseppl = [x for x in df_id.values if x not in true_opioid_id.values]
print(len(nouseppl))
print(nouseppl)
# These people have no opioid drugs data

17
['ID13362382498', 'ID14698922966', 'ID15002303111', 'ID19738321946', 'ID21430468895', 'ID27220994153', 'ID30313395070', 'ID36354083119', 'ID53521180550', 'ID55796310983', 'ID59083521579', 'ID62579261726', 'ID63293388003', 'ID77890885331', 'ID86806699161', 'ID96200812839', 'ID98715617553']


#### Why brand_name is not good filter:
Some brand_name contains non opioid drugs

In [482]:
opioid_all_time = rx_paid[rx_paid['PAY_DAY_SUPPLY_CNT'].notnull()]['brand_name'].value_counts()
print('total numbers of opioid generic_name', len(opioid_all_time))
mask = rx_paid['brand_name'].map(lambda x: x in opioid_all_time.index.values)
true_opioid_filtered_by_brand_name = rx_paid[mask]

total numbers of opioid generic_name 77


In [487]:
true_opioid_gn = true_opioid['generic_name'].drop_duplicates()
true_opioid_filtered_by_brand_name_gn = true_opioid_filtered_by_brand_name['generic_name'].drop_duplicates()
[x for x in true_opioid_filtered_by_brand_name_gn.values if x not in true_opioid_gn.values]

['BUTALBITAL-ACETAMINOPHEN-CAFFEINE TAB 50-325-40 MG',
 'BUTALBITAL-ASPIRIN-CAFFEINE CAP 50-325-40 MG',
 'BUTALBITAL-ACETAMINOPHEN-CAFFEINE CAP 50-325-40 MG',
 'BUTALBITAL-ACETAMINOPHEN-CAFFEINE CAP 50-300-40 MG',
 'MORPHINE SULFATE INJ 10 MG/ML',
 'HYDROMORPHONE HCL PRESERVATIVE FREE (PF) INJ 10 MG/ML',
 'MORPHINE SULFATE IV SOLN PF 10 MG/ML',
 'MORPHINE SULFATE POWDER',
 'NALTREXONE HCL (BULK) POWDER',
 'HYDROMORPHONE HCL POWDER',
 'BUTALBITAL-ASPIRIN-CAFFEINE TAB 50-325-40 MG',
 'FENTANYL SUBLINGUAL SPRAY 400 MCG',
 'TRAMADOL HCL (BULK) POWDER']

Other info

In [ ]:
# some special brands

'OPIUM' # only one data point

['MOVANTIK', # Opioid receptor antagonists
 'RELISTOR', # Opioid receptor antagonists
 'FENTANYL CITRATE', # OPIOID AGONISTS
 'HYDROMORPHONE HCL DOSETTE', # OPIOID AGONISTS
 'SYMPROIC'] # PERIPHERAL OPIOID RECEPTOR ANTAGONISTS

#### Why should  I use true_opioid but not opioid?
Because I can retain data of 85 more ppl

In [38]:
%%time
rx_paid['rx_cost'] = rx_paid['rx_cost'].map(lambda x: float(x))
true_opioid[['rx_cost','PAY_DAY_SUPPLY_CNT', 'PAYABLE_QTY', 'MME', 'generic_name']].isnull().apply(sum)

Wall time: 1.19 s


rx_cost                  0
PAY_DAY_SUPPLY_CNT    5348
PAYABLE_QTY           5348
MME                   6489
generic_name             0
dtype: int64

In [40]:
opioid[['PAY_DAY_SUPPLY_CNT', 'PAYABLE_QTY', 'MME', 'generic_name']].isnull().apply(sum)

PAY_DAY_SUPPLY_CNT      0
PAYABLE_QTY             0
MME                   499
generic_name            0
dtype: int64

In [41]:
len(opioid['id'].drop_duplicates())

13898

In [50]:
len(true_opioid[true_opioid['Days']<=0]['id'].drop_duplicates())

13983

In [52]:
13983-13898

85

## How should I fill missing values?

In [520]:
print('mode', true_opioid['PAY_DAY_SUPPLY_CNT'].mode())
print('mean',true_opioid['PAY_DAY_SUPPLY_CNT'].mean())
print('median',true_opioid['PAY_DAY_SUPPLY_CNT'].median())

mode 0    30.0
dtype: float64
mean 27.071109609505704
median 30.0


In [527]:
print('mode', true_opioid['PAYABLE_QTY'].mode())
print('mean',true_opioid['PAYABLE_QTY'].mean())
print('median',true_opioid['PAYABLE_QTY'].median())

mode 0    60.0
dtype: float64
mean 85.83430146175859
median 90.0


In [526]:
print('mode', true_opioid['MME'].mode())
print('mean',true_opioid['MME'].mean())
print('median',true_opioid['MME'].median())

mode 0    30.0
dtype: float64
mean 43.84522635140777
median 30.0


#### Fill PAY_DAY_SUPPLY_CNT with mode

In [78]:
mask_to_fill = true_opioid['PAY_DAY_SUPPLY_CNT'].isnull()
mode = true_opioid['PAY_DAY_SUPPLY_CNT'][~mask_to_fill].mode()
true_opioid['PAY_DAY_SUPPLY_CNT'][mask_to_fill] = [mode]*sum(mask_to_fill)
true_opioid[mask_to_fill].head()

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\IPython\core\interactiveshell.py:3325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ver

,id,event_descr,gpi_drug_class_description,rx_cost,net_paid_amount,brand_name,drug_group_description,generic_name,member_responsible_amount,gpi_drug_group8_id,Days,PAY_DAY_SUPPLY_CNT,PAYABLE_QTY,MME,DRUG_TYPE,Specialty,Specialty2,Specialty3
1386,ID1002482139,RX Claim - Paid,OPIOID AGONISTS,13.7,13.7,OXYCODONE HCL,PAIN,OXYCODONE HCL TAB 10 MG,0.00,65100075.0,472,30.0,NaN,NaN,NaN,NaN,NaN,NaN
3082,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,11.13,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,1.97,65991702.0,347,30.0,NaN,NaN,NaN,NaN,NaN,NaN
3107,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,11.13,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,1.97,65991702.0,379,30.0,NaN,NaN,NaN,NaN,NaN,NaN
3135,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,0,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,13.10,65991702.0,416,30.0,NaN,NaN,NaN,NaN,NaN,NaN
3159,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,0,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,13.10,65991702.0,445,30.0,NaN,NaN,NaN,NaN,NaN,NaN


#### Fill PAYABLE_QTY with mean

In [79]:
mask_to_fill = true_opioid['PAYABLE_QTY'].isnull()
mode = true_opioid['PAYABLE_QTY'][~mask_to_fill].mean()
true_opioid['PAYABLE_QTY'][mask_to_fill] = [mode]*sum(mask_to_fill)
true_opioid[mask_to_fill].head()

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,event_descr,gpi_drug_class_description,rx_cost,net_paid_amount,brand_name,drug_group_description,generic_name,member_responsible_amount,gpi_drug_group8_id,Days,PAY_DAY_SUPPLY_CNT,PAYABLE_QTY,MME,DRUG_TYPE,Specialty,Specialty2,Specialty3
1386,ID1002482139,RX Claim - Paid,OPIOID AGONISTS,13.7,13.7,OXYCODONE HCL,PAIN,OXYCODONE HCL TAB 10 MG,0.00,65100075.0,472,30.0,85.834301,NaN,NaN,NaN,NaN,NaN
3082,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,11.13,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,1.97,65991702.0,347,30.0,85.834301,NaN,NaN,NaN,NaN,NaN
3107,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,11.13,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,1.97,65991702.0,379,30.0,85.834301,NaN,NaN,NaN,NaN,NaN
3135,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,0,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,13.10,65991702.0,416,30.0,85.834301,NaN,NaN,NaN,NaN,NaN
3159,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,0,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,13.10,65991702.0,445,30.0,85.834301,NaN,NaN,NaN,NaN,NaN


In [525]:
#### Try to use linear regression to fill PAYABLE_QTY: more PAYABLE_QTY, more rx_cost
# 1. features: brand_names, rx_cost
# 2. labels: PAYABLE_QTY

# from patsy import dmatrices

# # encode brand_names
# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder(handle_unknown='ignore')
# brand_name = true_opioid['generic_name'].values.reshape(-1, 1)
# enc.fit(brand_name)

# X_brand = pd.DataFrame(enc.transform(brand_name).toarray())
# X_cost = true_opioid[['rx_cost']].reset_index(drop=True)
# y = true_opioid[['PAYABLE_QTY']].reset_index(drop=True)
# data = pd.concat([X_brand, X_cost, y], axis = 1, ignore_index=True)
# data.columns = enc.get_feature_names(['name']).tolist() + ['rx_cost', 'PAYABLE_QTY']

# y_tmp = pd.DataFrame([1]*data.shape[0])
# data_for_dmatrice = pd.concat([X_brand, X_cost, y_tmp], axis = 1, ignore_index=True)
# data_for_dmatrice.columns = enc.get_feature_names(['name']).tolist() + ['rx_cost', 'constant']

# formula = 'constant ~' + ' '.join(['rx_cost*Q("{}") +'.format(x) for x in enc.get_feature_names(['brand']).tolist()])[:-1]
# Y, X = dmatrices(formula, data_for_dmatrice, return_type='dataframe')

# train = true_opioid['PAYABLE_QTY'].reset_index(drop=True).notnull()
# trainy = true_opioid['PAYABLE_QTY'].notnull()
# tofill = true_opioid['PAYABLE_QTY'].reset_index(drop=True).isnull()

# X_train = X[train]
# y_train = true_opioid.loc[trainy, ['PAYABLE_QTY']]

# X_tofill = X[tofill]

# from sklearn.linear_model import LinearRegression
# linear = LinearRegression()
# linear.fit(X_train, y_train)
# linear.score(X_train, y_train)

# valuetofill = linear.predict(X_tofill)

# valuetofill_list = [x[0] for x in valuetofill]

# mask_to_fill = true_opioid['PAYABLE_QTY'].isnull()

# true_opioid['PAYABLE_QTY'][mask_to_fill] = valuetofill_list

# true_opioid[mask_to_fill].head()

PatsyError: Error evaluating factor: NameError: no data named 'brand_HYDROMORPHONE HCL TAB 2 MG' found
    constant ~rx_cost*Q("brand_ACETAMINOPHEN W/ CODEINE SOLN 120-12 MG/5ML") + rx_cost*Q("brand_ACETAMINOPHEN W/ CODEINE TAB 300-15 MG") + rx_cost*Q("brand_ACETAMINOPHEN W/ CODEINE TAB 300-30 MG") + rx_cost*Q("brand_ACETAMINOPHEN W/ CODEINE TAB 300-60 MG") + rx_cost*Q("brand_ACETAMINOPHEN-CAFFEINE-DIHYDROCODEINE CAP 320.5-30-16 MG") + rx_cost*Q("brand_BUPRENORPHINE HCL BUCCAL FILM 150 MCG (BASE EQUIVALENT)") + rx_cost*Q("brand_BUPRENORPHINE HCL BUCCAL FILM 300 MCG (BASE EQUIVALENT)") + rx_cost*Q("brand_BUPRENORPHINE HCL BUCCAL FILM 450 MCG (BASE EQUIVALENT)") + rx_cost*Q("brand_BUPRENORPHINE HCL BUCCAL FILM 600 MCG (BASE EQUIVALENT)") + rx_cost*Q("brand_BUPRENORPHINE HCL BUCCAL FILM 75 MCG (BASE EQUIVALENT)") + rx_cost*Q("brand_BUPRENORPHINE HCL BUCCAL FILM 750 MCG (BASE EQUIVALENT)") + rx_cost*Q("brand_BUPRENORPHINE HCL BUCCAL FILM 900 MCG (BASE EQUIVALENT)") + rx_cost*Q("brand_BUPRE

#### Fill MME with mode

In [80]:
mask_to_fill = true_opioid['MME'].isnull()
mode = true_opioid['MME'][~mask_to_fill].mode()
true_opioid['MME'][mask_to_fill] = [mode]*sum(mask_to_fill)
true_opioid[mask_to_fill].head()

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,event_descr,gpi_drug_class_description,rx_cost,net_paid_amount,brand_name,drug_group_description,generic_name,member_responsible_amount,gpi_drug_group8_id,Days,PAY_DAY_SUPPLY_CNT,PAYABLE_QTY,MME,DRUG_TYPE,Specialty,Specialty2,Specialty3
1386,ID1002482139,RX Claim - Paid,OPIOID AGONISTS,13.7,13.7,OXYCODONE HCL,PAIN,OXYCODONE HCL TAB 10 MG,0.00,65100075.0,472,30.0,85.834301,30.0,NaN,NaN,NaN,NaN
3082,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,11.13,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,1.97,65991702.0,347,30.0,85.834301,30.0,NaN,NaN,NaN,NaN
3107,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,11.13,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,1.97,65991702.0,379,30.0,85.834301,30.0,NaN,NaN,NaN,NaN
3135,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,0,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,13.10,65991702.0,416,30.0,85.834301,30.0,NaN,NaN,NaN,NaN
3159,ID10074598346,RX Claim - Paid,OPIOID COMBINATIONS,13.1,0,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,13.10,65991702.0,445,30.0,85.834301,30.0,NaN,NaN,NaN,NaN


## features_matthew_v2
supply_day, payable_qty, MME

In [267]:
# features3 = pd.read_csv('features_matthew_v2.csv')
# features3 = features3.set_index('key_0')
# features_matthew_v2 = features3.copy()
# features_matthew_v2.head()

In [268]:
%%time
######################################
# code to generate features_matthew_v2
######################################

opioid_grouped = true_opioid.groupby(by=['id'])
idtestlist = true_opioid['id'].drop_duplicates()

features3 = pd.DataFrame()
for ID in idtestlist:
    tmp = opioid_grouped.get_group(ID)

    # MME (per day) on day 0
    # Suuply_CNT on day 0
    on_day0 = tmp[tmp['Days'] == 0] 
    if not on_day0.empty:
        MME0 = on_day0['MME'].values[0]
        SC0 = on_day0['PAY_DAY_SUPPLY_CNT'].values[0]
        PQ0 = on_day0['PAYABLE_QTY'].values[0]
    else:
        MME0 = 0
        SC0 = 0
        PQ0 = 0

    # max MME (per day) prior to day 0
    # average MME (per day) prior to day 0
    # Total Supply_CNT prior to day 0
    prior_day0 = tmp[tmp['Days'] < 0]
    if not prior_day0.empty:
        maxMME = np.nanmax(prior_day0['MME'].values)
        avgMME = np.nanmean(prior_day0['MME'].values)
        totalSC = np.nansum(prior_day0['PAY_DAY_SUPPLY_CNT'].values)
        totalPQ = np.nansum(prior_day0['PAYABLE_QTY'].values)
    else:
        maxMME = 0
        avgMME = 0
        totalSC = 0
        totalPQ = 0

    output = pd.DataFrame({'MME_on_day0': MME0, 
                         'SUPPLY_CNT_on_day0': SC0,
                           'PAYABLE_QTY_on_day0': PQ0,
                         'max_MME_prior': maxMME,
                         'avg_MME_prior': avgMME,
                         'total_SUPPLY_CNT_prior': totalSC,
                          'total_PAYABLE_QTY_prior': totalPQ},
                          index = [ID])

    features3 = features3.append(output, sort=False)

# MME_on_day0, max_MME_prior, avg_MME_prior has some missing value, fill with medians
features3['MME_on_day0'] = features3['MME_on_day0'].fillna(np.nanmedian(features3['MME_on_day0']))
features3['max_MME_prior'] = features3['max_MME_prior'].fillna(np.nanmedian(features3['max_MME_prior']))
features3['avg_MME_prior'] = features3['avg_MME_prior'].fillna(np.nanmedian(features3['avg_MME_prior']))

# add one more feature: supply_times
supply_times = true_opioid[true_opioid['Days']<=0].groupby(by=['id'])['PAY_DAY_SUPPLY_CNT'].count()
supply_times = pd.DataFrame(supply_times)
supply_times.columns = ['supply_times']
features3 = features3.merge(supply_times, left_on=features3.index.values, right_on=supply_times.index.values)
features3 = features3.set_index('key_0')
# features3.to_csv('features_matthew_v2.csv')

Wall time: 2min 10s


In [269]:
# features_matthew_v2 = features3[[x for x in features3.columns if x not in ['max_MME_prior', 'avg_MME_prior', 
#                                                                            'PAYABLE_QTY_on_day0',
#                                                                           'total_PAYABLE_QTY_prior',
#                                                                           'MME_on_day0']]]
features_matthew_v2 = features3
features_matthew_v2.head()

,MME_on_day0,SUPPLY_CNT_on_day0,PAYABLE_QTY_on_day0,max_MME_prior,avg_MME_prior,total_SUPPLY_CNT_prior,total_PAYABLE_QTY_prior,supply_times
key_0,,,,,,,,
ID10010854159,15.0,5.0,15.0,0.0,0.000000,0.0,0.0,1
ID10013863216,10.0,90.0,180.0,0.0,0.000000,0.0,0.0,1
ID10024447278,50.0,3.0,20.0,22.5,20.000000,15.0,40.0,4
ID1002482139,60.0,30.0,120.0,90.0,37.254464,789.0,2895.0,29
ID1003386406,20.0,15.0,60.0,50.0,50.000000,3.0,20.0,2


In [270]:
features_matthew_v2.shape

(13983, 8)

## features_matthew_generic
Brand_name with PCA didn't work, so I tried generic_names with PCA.

In [271]:
%%time
############################################
# code to generate features_matthew_generic
############################################

opioid2_grouped = true_opioid[true_opioid['Days'] == 0].groupby(by=['id'])

idtestlist = true_opioid[true_opioid['Days'] == 0]['id'].drop_duplicates()

## those commented-out codes are used to get other entry values
# def product_sum(df):
#     return(df['MME'].values.dot(df['PAY_DAY_SUPPLY_CNT']e.values))

features = pd.DataFrame()
for ID in idtestlist:
    tmp = opioid2_grouped.get_group(ID)
    output = pd.DataFrame(tmp.groupby(by='generic_name')['PAY_DAY_SUPPLY_CNT'].agg(np.nansum)).T
#     output = output.iloc[0:1,:]
    output.index = [ID]
    # features = pd.concat([output, features], axis=1, sort=False)
    features = features.append(output, sort=False)

features = features.fillna(0)
# features.to_csv('features_matthew_generic_names.csv')

Wall time: 5min 2s


In [272]:
pca = PCA(n_components = 10) 
X10D = pca.fit_transform(features)
print(pca.explained_variance_ratio_)
sum(pca.explained_variance_ratio_)

[0.62794739 0.08259106 0.06615147 0.04182003 0.02824653 0.02293315
 0.02171731 0.01886087 0.01036229 0.00762625]


0.9282563380504804

In [273]:
# # tried to scale features so PCA can better condense it
# # StandardScaler and MaxMin both didn't work
# scaler = StandardScaler()
# scaler.fit(features)
# features_scale = scaler.transform(features)
# features_scale = pd.DataFrame(features_scale, index=features.index.values, columns=features.columns)

# pca = PCA(n_components = 10) 
# X10D = pca.fit_transform(features_scale)
# print(pca.explained_variance_ratio_)
# sum(pca.explained_variance_ratio_)

In [274]:
features_matthew_generic = pd.DataFrame(X10D, index=features.index.values)
features_matthew_generic.columns = ['generic_pc{}'.format(x) for x in features_matthew_generic.columns]
features_matthew_generic.head()

,generic_pc0,generic_pc1,generic_pc2,generic_pc3,generic_pc4,generic_pc5,generic_pc6,generic_pc7,generic_pc8,generic_pc9
ID10010854159,-5.971793,-1.614144,-2.656025,-1.964524,-0.980524,-0.237624,-0.197639,-1.948245,-0.518680,-0.377164
ID10013863216,78.735906,1.069676,1.642393,1.167419,0.471243,0.144658,0.077287,1.054565,0.264219,0.201644
ID10024447278,-11.004414,-1.851362,-3.089010,-2.442020,-1.466898,-0.723459,-1.952817,0.203388,-0.421221,-0.344819
ID1002482139,-11.091818,-1.960224,-3.295098,-2.648637,-1.491112,-0.421550,-0.412403,-3.473674,-2.260370,29.147746
ID1003386406,3.993819,-1.298400,-2.150329,-1.596060,-0.809728,-0.192650,-0.165295,-1.594973,-0.426575,-0.309069


In [275]:
features_matthew_generic.shape

(13979, 10)

#### pc_diagnosis

In [276]:
pc_diagnosis = pd.read_csv('pc_diagnosis.csv')
pc_diagnosis = pc_diagnosis.set_index('id')
pc_diagnosis.head()

,PC 1,PC 2,PC 3,PC 4,PC 5,PC 6,PC 7,PC 8,PC 9,PC 10
id,,,,,,,,,,
ID10010854159,0.008840,0.006021,0.001393,0.003137,-0.000543,-0.000012,-0.002715,0.000189,-0.002999,0.000369
ID10013863216,0.004783,-0.004110,0.009211,0.006220,-0.000302,-0.001218,-0.004039,0.003508,-0.007012,0.007118
ID10024447278,0.007123,0.018349,-0.002873,0.001940,-0.000999,0.001292,0.000691,-0.000391,-0.000094,-0.002040
ID1002482139,-0.003584,-0.004210,0.000235,-0.003246,-0.001197,0.002330,0.000364,-0.001483,0.000771,-0.001723
ID1003386406,0.001259,-0.000543,-0.000644,-0.003560,-0.000509,-0.003400,-0.002201,-0.008570,0.005852,0.004095


9 ppl have no diagnosis data

In [277]:
pc_diagnosis.shape

(13991, 10)

#### jake_features

In [278]:
# jake_features = pd.read_csv('jake_features.csv')
# jake_features = jake_features.set_index('id')

# # scaler = MinMaxScaler()
# # scaler.fit(jake_features)
# # jake_features_scale = scaler.transform(jake_features)
# # jake_features_scale = pd.DataFrame(jake_features_scale, index=jake_features.index.values, columns=jake_features.columns)

# # pca_jake = PCA(n_components = 4) 
# # X10D_jake = pca_jake.fit_transform(jake_features_scale)
# # print(pca_jake.explained_variance_ratio_) 

# # print(sum(pca_jake.explained_variance_ratio_))

# # jake_features = pd.DataFrame(X10D_jake, index=jake_features.index.values)
# # jake_features.columns = ['Jake_ '+ str(x) for x in jake_features.columns]
# jake_features = jake_features[['LIS LETTERS', 'COB DISCREPANCY']]
# jake_features.head()

#### Response variables

In [279]:
response_variable = pd.read_csv('response_variable_v2.csv')
response_variable = response_variable.dropna()
response_variable = response_variable.set_index('id')
response_variable.head()

,naive,LTOT_v2
id,,
ID10010854159,"[0, 706]",False
ID10013863216,[0],True
ID10024447278,[0],True
ID1002482139,[0],True
ID1003386406,[0],False


52 ppl have no PAY_SUPPLY_CNT data on days >= 0

In [280]:
response_variable.shape

(13948, 2)

#### Merge all features and response variables

In [281]:
df2 = pd.concat([response_variable[['LTOT_v2']], features_matthew_v2, pc_diagnosis, features_matthew_generic], 
                axis=1, join='inner')

df2.head()

,LTOT_v2,MME_on_day0,SUPPLY_CNT_on_day0,PAYABLE_QTY_on_day0,max_MME_prior,avg_MME_prior,total_SUPPLY_CNT_prior,total_PAYABLE_QTY_prior,supply_times,PC 1,...,generic_pc0,generic_pc1,generic_pc2,generic_pc3,generic_pc4,generic_pc5,generic_pc6,generic_pc7,generic_pc8,generic_pc9
ID10010854159,False,15.0,5.0,15.0,0.0,0.000000,0.0,0.0,1,0.008840,...,-5.971793,-1.614144,-2.656025,-1.964524,-0.980524,-0.237624,-0.197639,-1.948245,-0.518680,-0.377164
ID10013863216,True,10.0,90.0,180.0,0.0,0.000000,0.0,0.0,1,0.004783,...,78.735906,1.069676,1.642393,1.167419,0.471243,0.144658,0.077287,1.054565,0.264219,0.201644
ID10024447278,True,50.0,3.0,20.0,22.5,20.000000,15.0,40.0,4,0.007123,...,-11.004414,-1.851362,-3.089010,-2.442020,-1.466898,-0.723459,-1.952817,0.203388,-0.421221,-0.344819
ID1002482139,True,60.0,30.0,120.0,90.0,37.254464,789.0,2895.0,29,-0.003584,...,-11.091818,-1.960224,-3.295098,-2.648637,-1.491112,-0.421550,-0.412403,-3.473674,-2.260370,29.147746
ID1003386406,False,20.0,15.0,60.0,50.0,50.000000,3.0,20.0,2,0.001259,...,3.993819,-1.298400,-2.150329,-1.596060,-0.809728,-0.192650,-0.165295,-1.594973,-0.426575,-0.309069


In [282]:
df2.isnull().apply(sum)

LTOT_v2                    0
MME_on_day0                0
SUPPLY_CNT_on_day0         0
PAYABLE_QTY_on_day0        0
max_MME_prior              0
avg_MME_prior              0
total_SUPPLY_CNT_prior     0
total_PAYABLE_QTY_prior    0
supply_times               0
PC 1                       0
PC 2                       0
PC 3                       0
PC 4                       0
PC 5                       0
PC 6                       0
PC 7                       0
PC 8                       0
PC 9                       0
PC 10                      0
generic_pc0                0
generic_pc1                0
generic_pc2                0
generic_pc3                0
generic_pc4                0
generic_pc5                0
generic_pc6                0
generic_pc7                0
generic_pc8                0
generic_pc9                0
dtype: int64

In [283]:
# # Used to check missing value
# df2[df2['total_SUPPLY_CNT_prior'].isnull()]

# mask = (true_opioid['id'] == 'ID14009496762') & (true_opioid['Days']<=0)
# true_opioid[mask]

In [284]:
df2.shape

(13924, 29)

In [285]:
# Base line
df2['LTOT_v2'].value_counts()[False]/sum(df2['LTOT_v2'].value_counts())

0.505530020109164

In [286]:
X = df2.iloc[:,1:]
y = df2.iloc[:,0].map(lambda x: 1 if x == True else 0 )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

## Issue to be discussed: 16 ppl in holdout data has no supply_day values

In [287]:
# df_hould_out = pd.read_csv('C:/Users/spong/Desktop/Humana/HMAHCC_HOLDOUT.csv')
# opioid_hold_out = df_hould_out[df_hould_out['PAY_DAY_SUPPLY_CNT'].notnull()]
# len(opioid_hold_out['ID'].drop_duplicates())

## Models

### Logistics

In [288]:
%%time

logit_model = LogisticRegression()

cv_results = cross_validate(logit_model, X_train, y_train, cv=3,
                            scoring= 'roc_auc',
                            return_train_score=True)

print('train roc_auc: ', np.mean(cv_results['test_score']))

logit_model.fit(X_train, y_train)
prediction_on_X_test = logit_model.predict_proba(X_test)
print('test roc_auc: ', roc_auc_score(y_test, prediction_on_X_test[:,1]))

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


train roc_auc:  0.8599165754514301


C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


test roc_auc:  0.8738470422669701
Wall time: 1.79 s


### RF tuned on max_depth

In [289]:
%%time
parameters = {'max_depth': list(range(1,31,1)),
             'n_estimators': [100, 300]}
rf = RandomForestClassifier()
rf_grid = GridSearchCV(rf, parameters, scoring='roc_auc', cv=5)
rf_grid.fit(X_train, y_train)

rf_grid.best_params_

Wall time: 27min 10s


{'max_depth': 9, 'n_estimators': 300}

In [290]:
%%time
rf_model = RandomForestClassifier(n_estimators=rf_grid.best_params_['n_estimators'], 
                                  max_depth=rf_grid.best_params_['max_depth'],
                                  random_state=100)

cv_results = cross_validate(rf_model, X_train, y_train, cv=3,
                            scoring= 'roc_auc',
                            return_train_score=True,
                            return_estimator =True)

print('train roc_auc: ', np.mean(cv_results['test_score']))

rf_model.fit(X_train, y_train)
prediction_on_X_test = rf_model.predict_proba(X_test)
print('test roc_auc: ', roc_auc_score(y_test, prediction_on_X_test[:,1]))

train roc_auc:  0.8793827982587823
test roc_auc:  0.8938360005486757
Wall time: 13.2 s


In [291]:
# parameters we can use to tune RF
# parameters = {'n_estimators':list(range(100,2100,100)),
#               'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#               'min_samples_split': [2, 5, 10],
#               'min_samples_leaf': [1, 2, 4],
#               'max_features': ['auto', 'sqrt']}

#### RF features importance

In [292]:
for idx,estimator in enumerate(cv_results['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = X.columns,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print('===================================')
    print(feature_importances)

Features sorted by their score for estimator 0:
                         importance
SUPPLY_CNT_on_day0         0.178811
total_PAYABLE_QTY_prior    0.087390
total_SUPPLY_CNT_prior     0.080274
generic_pc9                0.080071
PAYABLE_QTY_on_day0        0.065814
supply_times               0.049305
PC 5                       0.040781
generic_pc8                0.040602
PC 6                       0.035146
avg_MME_prior              0.024562
generic_pc6                0.024513
generic_pc5                0.022967
PC 4                       0.021838
PC 7                       0.019084
PC 3                       0.018908
max_MME_prior              0.018630
PC 10                      0.017803
generic_pc7                0.017802
PC 9                       0.016967
generic_pc0                0.016734
PC 2                       0.016680
MME_on_day0                0.016483
PC 8                       0.016353
generic_pc4                0.016017
PC 1                       0.015729
generic_pc3     

#### Code to visualize the effect of different hyperparameters of RF

In [293]:
# grid = list(range(1,31,1))
# train_accuracy = list()
# test_accuracy = list()
# for i in grid:
#     rf_model = RandomForestClassifier(n_estimators=100, max_depth=i,
#                                       random_state=0)
#     cv_results = cross_validate(rf_model, X_train, y_train, cv=3,
#                             scoring= 'accuracy',
#                             return_train_score=True)

#     train_accuracy.append(np.mean(cv_results['train_score']))
#     test_accuracy.append(np.mean(cv_results['test_score']))

# test_res = pd.DataFrame([train_accuracy, test_accuracy], index = ['train_accuracy', 'test_accuracy']).T
# test_res.set_index = grid
# test_res.plot()
# plt.ylabel('Accuracy')
# plt.xlabel('max_depth')

### KNN classifier on tuned K 

In [294]:
# %%time
# parameters = {'n_neighbors': list(range(5,205,10))}
# neigh = KNeighborsClassifier()
# neigh_grid = GridSearchCV(neigh, parameters, scoring='roc_auc', cv=5)
# neigh_grid.fit(X_train, y_train)

# neigh_grid.best_params_

In [295]:
# %%time
# neigh = KNeighborsClassifier(n_neighbors=25)

# cv_results = cross_validate(neigh, X_train, y_train, cv=3,
#                             scoring= 'roc_auc',
#                             return_train_score=True,
#                            return_estimator =True)

# print('train roc_auc: ', np.mean(cv_results['test_score']))

# neigh.fit(X_train, y_train)
# prediction_on_x_test = neigh.predict(X_test)
# print('test roc_auc: ', roc_auc_score(y_test, prediction_on_x_test))

### Neural Network

In [296]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scale = scaler.transform(X)
y_array = y.values
X_train, X_test_s, y_train, y_test = train_test_split(X_scale, y_array, test_size=0.33, random_state=33)
X_train_mlp, X_validation, y_train_mlp, y_validation = train_test_split(X_train, y_train, test_size=0.15, random_state=33)

In [297]:
%%time
model = Sequential()

#First Hidden Layer
model.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=X_train.shape[1]))

#Second  Hidden Layer
model.add(Dense(64, activation='relu', kernel_initializer='random_normal'))

# #Third  Hidden Layer
# model.add(Dense(16, activation='relu', kernel_initializer='random_normal'))

#Output Layer
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# there's no easy way to incorporate roc_auc in MLP's training process
# so I use accuracy, which can be a proxy of roc_auc, to train models
# then use roc_auc to assess the prediction on test set

# add early stop to prevent overfitting
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_model2.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train, y_train,
                    validation_split = 0.15,
                    epochs=4000, verbose=0, callbacks=[es, mc])

saved_model = load_model('best_model2.h5')

_, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
_, test_acc = saved_model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


Epoch 00001: val_loss improved from inf to 0.49700, saving model to best_model2.h5

Epoch 00002: val_loss improved from 0.49700 to 0.48811, saving model to best_model2.h5

Epoch 00003: val_loss improved from 0.48811 to 0.47530, saving model to best_model2.h5

Epoch 00004: val_loss improved from 0.47530 to 0.47282, saving model to best_model2.h5

Epoch 00005: val_loss improved from 0.47282 to 0.46499, saving model to best_model2.h5

Epoch 00006: val_loss improved from 0.46499 to 0.46381, saving model to best_model2.h5

Epoch 00007: val_loss did not improve from 0.46381

Epoch 00008: val_loss did not improve from 0.46381

Epoch 00009: val_loss improved from 0.46381 to 0.46138, saving model to best_model2.h5

Epoch 00010: val_loss did not improve from 0.46138

Epoch 00011: val_loss improved from 0.46138 to 0.46074, saving model to best_model2.h5

Epoch 00012: val_loss did not improve from 0.46074

Epoch 00013: val_loss improved from 0.46074 to 0.45822, saving model to best_model2.h5

Epo

In [298]:
prediction_on_X_train = saved_model.predict_proba(X_train)
prediction_on_X_train = [x[0] for x in prediction_on_X_train]
print('roc_auc on train: ', roc_auc_score(y_train, prediction_on_X_train))

prediction_on_X_test = saved_model.predict_proba(X_test_s)
prediction_on_X_test = [x[0] for x in prediction_on_X_test]
print('roc_auc on test: ', roc_auc_score(y_test, prediction_on_X_test))

roc_auc on train:  0.8947687282398186
roc_auc on test:  0.8952407503821406


In [299]:
# reserve best performance as of Oct 2nd

# prediction_on_X_train = saved_model.predict_proba(X_train)
# prediction_on_X_train = [x[0] for x in prediction_on_X_train]
# print('roc_auc on train: ', roc_auc_score(y_train, prediction_on_X_train))

# prediction_on_X_test = model.predict_proba(X_test)
# prediction_on_X_test = [x[0] for x in prediction_on_X_test]
# print('roc_auc on test: ', roc_auc_score(y_test, prediction_on_X_test))

### LightGBM

In [300]:
import lightgbm as lgb

X = df2.iloc[:,1:]
y = df2.iloc[:,0].map(lambda x: 1 if x == True else 0 )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

In [301]:
%%time
bst = lgb.LGBMClassifier(boosting_type= 'gbdt',
                        objective = 'binary',
                        max_depth = 3,
                        n_estimators = 1000)

bst.fit(X_train.values, y_train.values)

prediction_on_X_train = bst.predict_proba(X_train)
print('train roc_auc: ', roc_auc_score(y_train, prediction_on_X_train[:,1]))

prediction_on_X_test = bst.predict_proba(X_test)
print('test roc_auc: ', roc_auc_score(y_test, prediction_on_X_test[:,1]))

train roc_auc:  0.9755976060811719
test roc_auc:  0.8908864900749579
Wall time: 694 ms


#### Combine NN, RF and LightGB

In [302]:
NN_predict = saved_model.predict_proba(X_test_s)
bst_predict = bst.predict_proba(X_test)
rf_predict = rf_model.predict_proba(X_test)

nn_prob = np.array([ x[0] for x in NN_predict])
bst_prob = bst_predict[:,1]
rf_prob = rf_predict[:,1]

avg = (nn_prob+bst_prob+rf_prob)/3
avg

print('test roc_auc: ', roc_auc_score(y_test, avg))

test roc_auc:  0.8997427893487205
